In [119]:
%pip install eep153_tools
%pip install python_gnupg
%pip install -U gspread_pandas
#load in file from class
def format_id(id,zeropadding=0):
    """Nice string format for any id, string or numeric.

    Optional zeropadding parameter takes an integer
    formats as {id:0z} where
    """
    if pd.isnull(id) or id in ['','.']: return None

    try:  # If numeric, return as string int
        return ('%d' % id).zfill(zeropadding)
    except TypeError:  # Not numeric
        return id.split('.')[0].strip().zfill(zeropadding)
    except ValueError:
        return None

data_url = "https://docs.google.com/spreadsheets/d/1GTo423_gUJe1Von9jypWAbC0zSQ7WGegAWPuRi7eJAI/edit?gid=1410082681#gid=1410082681"

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [120]:
import pandas as pd
import re 
from eep153_tools.sheets import read_sheets

#create recipes df
recipes = read_sheets(data_url, sheet="recipes")
recipes = (recipes
           .assign(parent_foodcode = lambda df: df["parent_foodcode"].apply(format_id),
                   ingred_code = lambda df: df["ingred_code"].apply(format_id))
           .rename(columns={"parent_desc": "recipe"}))
recipes.head()


,parent_foodcode,recipe,ingred_code,ingred_desc,ingred_wt
0,11340000,"Imitation milk, non-soy, sweetened",43543,"Milk, imitation, non-soy",100.0
1,11460150,"Yogurt, frozen, NS as to flavor, lowfat milk",1298,"Yogurt, frozen, flavors other than chocolate, ...",100.0
2,11460160,"Yogurt, frozen, chocolate, lowfat milk",1117,"Yogurt, plain, low fat, 12 grams protein per 8...",81.8
3,11460160,"Yogurt, frozen, chocolate, lowfat milk",19166,"Cocoa, dry powder, unsweetened, processed with...",5.2
4,11460160,"Yogurt, frozen, chocolate, lowfat milk",19335,"Sugars, granulated",13.0


In [121]:
#List of non-vegan keywords AND non-natural foods keywords (including frozen, processed, etc).
NON_VEGAN_KEYWORDS = [
    "beef", "pork", "chicken", "turkey", "fish", "seafood", "shellfish", "shrimp", "crab","crabs",
    "lamb", "goat", "duck", "goose", "tuna", "salmon", "cod", "bacon", "ham",
    "shellfish", "lobster", "mussels", "oysters", "scallops", "octopus", "eel",
    "organ meat", "milk","Eggnog" "cheese", "butter", "cream","ice cream", "yogurt", "whey",
    "casein", "lactose", "ghee", "buttermilk", "egg", "eggs", "mayo", "mayonnaise", "albumen",
    "albumin", "lysozyme", "ovomucoid", "ovomucin", "ovovitellin", "honey",
    "bee pollen", "royal jelly", "propolis", "shellac", "confectioner’s glaze",
    "carmine", "cochineal", "lard", "tallow", "suet", "gelatin", "collagen",
    "isinglass", "bone broth", "bone stock", "fish sauce", "oyster sauce",
    "shrimp paste", "worcestershire sauce", "anchovies", "rennet", "pepsin",
    "bone char", "vitamin d3", "lanolin", "omega-3 fish oil", "caseinate",
    "lecithin (egg)", "cysteine", "l-cysteine", "glycerin (animal)",
    "glycerol (animal)", "stearic acid (animal)", "tallowate", "sodium tallowate",
    "capric acid", "caprylic acid", "cheese", "pudding", "processed", "veal",'sirloin', "steak", "animal",
    "Custard", "Mousse", "chocolate", "Meatballs", "meat", "Gravy", "poultry","baby", "frozen", 'dairy', 'lump',"peas","school"
]

#this partal match: "milkshake" or "eggroll" will get flagged (since "milk" or "egg" is in the keyword list).
NON_VEGAN_PATTERN = re.compile(
    '|'.join(map(re.escape, NON_VEGAN_KEYWORDS)),
    re.IGNORECASE
)

def filter_vegan_ingredients(df: pd.DataFrame) -> pd.DataFrame:
    # 1) Convert to string, lowercase, remove punctuation
    df["recipe"] = df["recipe"].astype(str).str.lower().fillna("")
    df["recipe"] = df["recipe"].str.replace(r"[^\w\s]", "", regex=True)

    df["ingred_desc"] = df["ingred_desc"].astype(str).str.lower().fillna("")
    df["ingred_desc"] = df["ingred_desc"].str.replace(r"[^\w\s]", "", regex=True)

    # 2) Create a mask for rows that do NOT contain non-vegan keywords
    mask = ~(df["recipe"].str.contains(NON_VEGAN_PATTERN, na=False, regex=True) |
             df["ingred_desc"].str.contains(NON_VEGAN_PATTERN, na=False, regex=True))

    return df[mask]

In [122]:
#start copying code from mini lecture VEGAN

#create nutrition df
nutrition = (read_sheets(data_url, sheet="nutrients")
             .assign(ingred_code = lambda df: df["ingred_code"].apply(format_id)))

display(nutrition.head())
nutrition.columns
nutrition.shape



# normalize weights to percentage terms. 
vegan_recipes['ingred_wt'] = vegan_recipes['ingred_wt']/vegan_recipes.groupby(['parent_foodcode'])['ingred_wt'].transform("sum")

# we're going to extend the recipes data frame to include the nutrient profiles of its ingredients (in 100g)
df_vegan = vegan_recipes.merge(nutrition, how="left", on="ingred_code")

# multiply all nutrients per 100g of an ingredient by the weight of that ingredient in a recipe.
numeric_cols = list(df_vegan.select_dtypes(include=["number"]).columns)
numeric_cols.remove("ingred_wt")
df_vegan[numeric_cols] = df_vegan[numeric_cols].mul(df_vegan["ingred_wt"], axis=0)

# sum nutrients of food codes (over the multiple ingredients)
# python tip: one can merge dictionaries dict1 dict2 using **, that is: dict_merge = {**dict1, **dict2}. The ** effectively "unpacks" the key value pairs in each dictionary
df_vegan = df_vegan.groupby('parent_foodcode').agg({**{col: "sum" for col in numeric_cols},
                                        "recipe": "first"})

df_vegan.index.name = "recipe_id"

food_names = df_vegan["recipe"]
print(food_names.head())
df_vegan.head(10)
df_vegan.shape

,ingred_code,Ingredient description,Capric acid,Lauric acid,Myristic acid,Palmitic acid,Palmitoleic acid,Stearic acid,Oleic acid,Linoleic Acid,...,Vitamin B12,"Vitamin B-12, added",Vitamin B6,Vitamin C,Vitamin D,Vitamin E,"Vitamin E, added",Vitamin K,Water,Zinc
0,1001,"Butter, salted",2.529,2.587,7.436,21.697,0.961,9.999,19.961,2.728,...,0.17,0.0,0.003,0.0,0.0,2.32,0.0,7.0,15.87,0.09
1,1002,"Butter, whipped, with salt",2.039,2.354,7.515,20.531,1.417,7.649,17.370,2.713,...,0.07,0.0,0.008,0.0,0.0,1.37,0.0,4.6,16.72,0.05
2,1003,"Butter oil, anhydrous",2.495,2.793,10.005,26.166,2.228,12.056,25.026,2.247,...,0.01,0.0,0.001,0.0,0.0,2.80,0.0,8.6,0.24,0.01
3,1004,"Cheese, blue",0.601,0.491,3.301,9.153,0.816,3.235,6.622,0.536,...,1.22,0.0,0.166,0.0,0.5,0.25,0.0,2.4,42.41,2.66
4,1005,"Cheese, brick",0.585,0.482,3.227,8.655,0.817,3.455,7.401,0.491,...,1.26,0.0,0.065,0.0,0.5,0.26,0.0,2.5,41.11,2.60


recipe_id
11115400    kefir ns as to fat content
11440060                  tzatziki dip
11551050             licuado or batido
11553100            fruit smoothie nfs
11710000            infant formula nfs
Name: recipe, dtype: object


(3893, 66)

In [123]:
prices = read_sheets(data_url, sheet="prices")[["food_code", "year", "price"]]

prices["food_code"] = prices["food_code"].apply(format_id)

prices = prices.set_index(["year", "food_code"])
print(prices.index.levels[0])

# we'll focus on the latest price data
prices = prices.xs("2017/2018", level="year")

# drop rows of prices where the price is "NA"
prices = prices.dropna(subset="price")

print(f"We have prices for {prices.shape[0]} unique recipes (FNDDS food codes)")

Index(['2011/2012', '2013/2014', '2015/2016', '2017/2018'], dtype='object', name='year')
We have prices for 4435 unique recipes (FNDDS food codes)


In [124]:
#add diet requirements

rda = read_sheets(data_url, sheet="rda")

rda = rda.set_index("Nutrient")
rda_min = rda[rda["Constraint Type"].isin(["RDA", "AI"])].copy()


rda.columns, rda.head(), rda_min.head()

(Index(['Nutrient Type', 'Unit', 'Constraint Type', 'Female_19_30',
        'Female_endurance_athlete', 'Male_19_30', 'Male_endurance_athlete'],
       dtype='object'),
               Nutrient Type  Unit Constraint Type  Female_19_30  \
 Nutrient                                                          
 Energy                Macro  kcal             RDA        2000.0   
 Protein               Macro     g             RDA          46.0   
 Carbohydrate          Macro     g             RDA         130.0   
 Dietary Fiber         Macro     g             RDA          28.0   
 Linoleic Acid         Macro     g              AI          12.0   
 
                Female_endurance_athlete  Male_19_30  Male_endurance_athlete  
 Nutrient                                                                     
 Energy                           2800.0      2400.0                  3500.0  
 Protein                            80.0        56.0                   112.0  
 Carbohydrate                      48

In [8]:
common_recipes = df_vegan.index.intersection(prices.index)

# python tip: given a list of indices, "loc" both subsets and sorts. 
df_vegan = df_vegan.loc[common_recipes]
prices = prices.loc[common_recipes]

# lets remap the price dataframe index to be the actual food names.
prices.index = prices.index.map(food_names)

A_all = df_vegan.T

print(prices.head())
print(A_all.head())

                                                       price
kefir ns as to fat content                          0.345625
tzatziki dip                                        1.217789
licuado or batido                                   0.189099
fruit smoothie nfs                                  0.462558
infant formula readytofeed similac expert care ...  1.074909
                 11115400  11440060  11551050  11553100 11710051 11710055  \
Capric acid        0.0195  0.005673   0.00048  0.000813    0.689    0.689   
Lauric acid         0.026  0.000273  0.000959  0.001626    0.023    0.023   
Myristic acid      0.0945  0.000545  0.000959  0.001626    0.007    0.007   
Palmitic acid      0.2805    5.7604  0.053557  0.083665    0.137    0.137   
Palmitoleic acid   0.0185  0.638909  0.005183  0.008129    0.003    0.003   

                  11710357  11710358 11710377 11710378  ... 95312410 95312560  \
Capric acid       0.078597  0.078597      0.0      0.0  ...      0.0      0.0   
Lauric ac

In [125]:
# pick a demographic (column from rda dataframe)
'''
select from 
['Female_19_30', 'Female_endurance_athlete', 'Male_19_30', 'Male_endurance_athlete']
'''
group = "Male_19_30"

# create lower bounds and upper bounds.
bmin = rda.loc[rda['Constraint Type'].isin(['RDA', 'AI']), group]
bmax = rda.loc[rda['Constraint Type'].isin(['UL']), group]

# reindex ensures we only keep nutrients in bmin/bmax
Amin = A_all.reindex(bmin.index).dropna(how='all')
Amax = A_all.reindex(bmax.index).dropna(how='all')

b = pd.concat([bmin, -bmax])
A = pd.concat([Amin, -Amax])

#python tip: by typing "=" after the variable name inside the curly braces, it formats the output so we don't have to write f"variable = {variable}"
print(f"{bmin.shape=}")
print(f"{Amin.shape=}")
print(f"{bmax.shape=}")
print(f"{Amax.shape=}")
print(f"{b.shape=}")
print(f"{A.shape=}")
print(f"{prices.shape=}")

bmin.shape=(26,)
Amin.shape=(26, 1834)
bmax.shape=(1,)
Amax.shape=(1, 1834)
b.shape=(27,)
A.shape=(27, 1834)
prices.shape=(4435, 1)


In [126]:
from  scipy.optimize import linprog as lp
import numpy as np
p = prices
tol = 1e-6 # Numbers in solution smaller than this (in absolute value) treated as zeros
result = lp(p, -A, -b, method='highs')


ValueError: Invalid input for linprog: A_ub must have exactly two dimensions, and the number of columns in A_ub must be equal to the size of c

In [127]:
print(f"Cost of diet for a vegan {group} is ${result.fun:.2f} per day.")

Cost of diet for a vegan Male_19_30 is $4.04 per day.


In [12]:
# lets mess with the index on price df so they are recipe names not ids.

# get the result x in a series with food names
diet = pd.Series(result.x,index=prices.index)


print(f"Cost of diet for a vegan {group} is ${result.fun:.2f} per day. \n")

print(f"As a vegan {group} you'll be eating (in 100s of grams or milliliters): \n")

print(round(diet[diet >= tol], 2))

Cost of diet for a vegan Male_endurance_athlete is $4.04 per day. 

As a vegan Male_endurance_athlete you'll be eating (in 100s of grams or milliliters): 

peruvian beans from dried        1.79
vermicelli made from soybeans    3.21
peanuts unroasted                0.90
cereal rice flakes               0.74
cereal toasted oat               1.32
beans and rice with tomatoes     7.50
cilantro raw                     0.22
corn oil                         0.23
sugar nfs                        0.58
dtype: float64


In [14]:
#Function for deliverable [A] Dietary Reference Intakes

#NEED TO ADD MORE categories into the Google sheet to make better, females, males of different ages, etc.
def get_population_dri(population, rda_df) -> pd.Series:
    
    # 1. Filter rows to only those where Constraint Type is RDA or AI
    rda_filtered = rda_df[rda_df["Constraint Type"].isin(["RDA", "AI"])].copy()

    # 3. Extract the column for the chosen population as a Series
    dri_series = rda_filtered[population]

    # 4. Drop any rows that are NaN (just in case)
    dri_series.dropna(inplace=True)

    # 5. Return the final Series
    return dri_series


In [15]:
#example of get_population_dri function

'''
select population from 
['Female_19_30', 'Female_endurance_athlete', 'Male_19_30', 'Male_endurance_athlete']
'''

population = "Female_endurance_athlete"
dri_for_female_19_30 = get_population_dri(population, rda)

print("Dietary recommendations (RDA) for", population)
print(dri_for_female_19_30)

Dietary recommendations (RDA) for Female_endurance_athlete
Nutrient
Energy            2800.0
Protein             80.0
Carbohydrate       488.0
Dietary Fiber       28.0
Linoleic Acid       12.0
Linolenic Acid       1.1
Calcium           1000.0
Iron                22.0
Magnesium          310.0
Phosphorus         700.0
Potassium         4700.0
Zinc                 8.0
Copper               0.9
Selenium            55.0
Vitamin A          700.0
Vitamin E           15.0
Vitamin D           25.0
Vitamin C           75.0
Thiamin              1.1
Riboflavin           1.1
Niacin              14.0
Vitamin B6           1.3
Vitamin B12          2.4
Choline            425.0
Vitamin K           90.0
Folate             400.0
Name: Female_endurance_athlete, dtype: float64


In [97]:
import pandas as pd
import numpy as np
from scipy.optimize import linprog

def min_cost_diet_with_remapped_names(
    population: str,
    rda: pd.DataFrame,
    df_vegan: pd.DataFrame,
    prices: pd.DataFrame,
    name_column: str = "desc"
) -> pd.Series:

    # 1) Intersect df_vegan & prices so they match on numeric codes
    common_recipes = df_vegan.index.intersection(prices.index)
    df_vegan = df_vegan.loc[common_recipes]
    prices = prices.loc[common_recipes]

    # 2) Create a mapping from code -> name
    food_names_map = df_vegan[name_column].to_dict()

    # 3) Remap the index of 'prices' to be the actual food names
    prices.index = prices.index.map(food_names_map)
    df_vegan.index = df_vegan.index.map(food_names_map)

    # 4) Build the linear program for RDA constraints
    # Filter rda for RDA or AI
    rda_min = rda[rda["Constraint Type"].isin(["RDA", "AI"])].copy()

    if population not in rda_min.columns:
        raise ValueError(f"Population '{population}' not in RDA columns: {rda_min.columns.tolist()}")

    # Series of required amounts for each nutrient
    bmin = rda_min[population].dropna()

    # For each nutrient in bmin, check df_vegan columns
    nutrients = bmin.index.tolist()
    for nut in nutrients:
        if nut not in df_vegan.columns:
            raise ValueError(f"Nutrient '{nut}' not found in df_vegan columns.")

    # Convert >= constraints to <= form
    A_bmin = df_vegan[nutrients].to_numpy()     
    A_ub = -A_bmin.T                             
    b_ub = -bmin.values                          

    # cost vector
    if "price" not in prices.columns:
        raise ValueError("The 'prices' DataFrame must have a 'price' column.")
    cost_vector = prices["price"].to_numpy()

    bounds = [(0, None)] * len(df_vegan)

    # Solve LP
    result = linprog(
        c=cost_vector,
        A_ub=A_ub,
        b_ub=b_ub,
        bounds=bounds,
        method="highs"
    )
    if not result.success:
        raise RuntimeError(f"Linear program failed: {result.message}")


    # 6) Print only the cost & the foods in 100s of grams/mL
    
    print(f"Cost of diet for a vegan {population} is ${result.fun:.2f} per day.\n")

    diet = pd.Series(result.x, index=df_vegan.index, name="servings")
    nonzero = diet[diet > 1e-6].copy()

    # Convert to "100s of grams/mL"
    diet_100 = (nonzero * 100).round(2)

    print(f"As a vegan {population}, you'll be eating (in 100s of grams/mL):")
    # Use .to_string() to avoid printing "Name: servings, dtype: float64"
    print(diet_100.to_string())

    return result


In [104]:
min_cost_diet_with_remapped_names(
    "Male_19_30",
    rda,
    df_vegan,
    prices,
    "recipe"   
)

Cost of diet for a vegan Male_19_30 is $3.49 per day.

As a vegan Male_19_30, you'll be eating (in 100s of grams/mL):
peruvian beans from dried                        1152.21
cereal toasted oat                                 49.03
nutritional powder mix high protein herbalife      92.15


        message: Optimization terminated successfully. (HiGHS Status 7: Optimal)
        success: True
         status: 0
            fun: 3.486593629389795
              x: [ 0.000e+00  0.000e+00 ...  0.000e+00  0.000e+00]
            nit: 5
          lower:  residual: [ 0.000e+00  0.000e+00 ...  0.000e+00
                              0.000e+00]
                 marginals: [ 2.137e-01  1.188e+00 ...  8.657e-02
                              4.821e-01]
          upper:  residual: [       inf        inf ...        inf
                                    inf]
                 marginals: [ 0.000e+00  0.000e+00 ...  0.000e+00
                              0.000e+00]
          eqlin:  residual: []
                 marginals: []
        ineqlin:  residual: [ 3.439e+02  9.670e+01 ...  9.277e+01
                              1.344e+03]
                 marginals: [-0.000e+00 -0.000e+00 ... -0.000e+00
                             -0.000e+00]
 mip_node_count: 0
 mip_dual_bound: 0.0
        mip_g